In [1]:
import pandas
import json

In [2]:
def get_dictionary(path):
    with open(path, 'r') as file:
        return json.load(file)

In [3]:
def get_data_frame(dictionary):
    rows = []
    for benchmark_dictionary in dictionary['benchmarks']:
        name = benchmark_dictionary['metadata']['name']
    
        for run_dictionary in benchmark_dictionary['runs']:
            duration = run_dictionary['metadata']['duration']
            rows.append(dict(name=name, duration=duration))
    
    return pandas.DataFrame(rows)

In [4]:
names = ['baseline', 'v4.1.2', 'sub', 'subopt']

data_frames = {name: get_data_frame(get_dictionary(f'./{name}.json')) for name in names}

data_frame = pandas.concat({name: data_frame.groupby('name').mean() for name, data_frame in data_frames.items()}).unstack(0)
data_frame.columns = names
data_frame.iloc[:, 1:] = data_frame.iloc[:, 1:].apply(lambda series: series / data_frame.iloc[:, 0])
data_frame = data_frame.sort_values([data_frame.columns[data_frame.iloc[:, 1:].min().argmin() + 1], 'name'], ascending=[False, True])

def _(data_frame):
    return data_frame.to_string(float_format=lambda float: f'{float:0.2f}')
    
def _(data_frame):
    return data_frame.to_markdown(floatfmt='0.2f')

print(_(data_frame))

| name                       |   baseline |   v4.1.2 |    sub |   subopt |
|:---------------------------|-----------:|---------:|-------:|---------:|
| raytrace                   |       1.34 |    26.01 |  29.02 |    20.58 |
| scimark_sor                |       0.55 |    26.13 |  27.74 |    20.08 |
| scimark_lu                 |       0.55 |    24.15 |  25.27 |    18.99 |
| go                         |       0.63 |    30.64 |  27.46 |    18.16 |
| generators                 |       0.88 |    18.51 |  24.18 |    16.12 |
| spectral_norm              |       0.47 |    19.47 |  20.65 |    14.97 |
| pyflate                    |       1.96 |    18.55 |  16.60 |    11.75 |
| richards                   |       0.45 |    16.67 |  16.35 |    11.56 |
| richards_super             |       0.54 |    15.32 |  15.37 |    10.58 |
| pprint_pformat             |       6.59 |    15.44 |  12.09 |     7.89 |
| pprint_safe_repr           |       3.22 |    15.29 |  11.77 |     7.52 |
| nbody                  